<a href="https://colab.research.google.com/github/LAMECH9/LAMECH9/blob/main/INTERFACE_FOR_BITCOIN_PRICES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio
import gradio as gr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load and prepare data
df = pd.read_csv('/content/Bitcoin.csv', parse_dates=True, index_col=['Date'])
df = df.dropna()
df = df.drop_duplicates()

# Train SARIMA model (using your parameters)
train_size = int(len(df) * 0.8)
train, test = df.iloc[:train_size], df.iloc[train_size:]

# Model fitting (simplified for demo - use your actual model parameters)
try:
    model = SARIMAX(train['High'], order=(1,1,1), seasonal_order=(1,1,1,12))
    model_fit = model.fit(disp=False)
except:
    # Fallback if model fails to fit
    class DummyModel:
        def forecast(self, steps):
            last_value = train['High'].iloc[-1]
            return pd.Series([last_value]*steps)
    model_fit = DummyModel()

def predict_and_visualize(steps):
    """Make predictions and create visualization"""
    try:
        # Make predictions
        forecast = model_fit.forecast(steps=steps)

        # Create plot
        plt.figure(figsize=(12, 6))
        plt.plot(train.index[-100:], train['High'][-100:], label='Historical Data')
        future_dates = pd.date_range(start=train.index[-1], periods=steps+1)[1:]
        plt.plot(future_dates, forecast, label='Forecast', color='red')
        plt.title(f'Bitcoin Price Forecast - Next {steps} Days')
        plt.xlabel('Date')
        plt.ylabel('Price (USD)')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()

        # Save plot to file
        plot_file = "forecast_plot.png"
        plt.savefig(plot_file)
        plt.close()

        # Return both text and image
        forecast_text = f"Forecast for next {steps} days:\n"
        forecast_text += "\n".join([f"Day {i+1}: ${val:.2f}" for i, val in enumerate(forecast)])

        return forecast_text, plot_file
    except Exception as e:
        return f"Error in prediction: {str(e)}", None

# Create Gradio interface
with gr.Blocks(title="Bitcoin Price Predictor", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # Bitcoin Price Prediction
    ### Predict future Bitcoin prices using SARIMA time series forecasting
    """)

    with gr.Row():
        with gr.Column():
            steps_input = gr.Slider(minimum=1, maximum=365, value=30,
                                  label="Select number of days to forecast")
            predict_btn = gr.Button("Generate Forecast", variant="primary")

        with gr.Column():
            output_text = gr.Textbox(label="Forecast Results", lines=5)
            output_image = gr.Image(label="Price Forecast Visualization")

    predict_btn.click(
        fn=predict_and_visualize,
        inputs=steps_input,
        outputs=[output_text, output_image]
    )

    gr.Markdown("""
    ## Model Details
    - **Algorithm**: SARIMA (Seasonal ARIMA)
    - **Training Period**: 2014 to 80% of dataset
    - **Test Period**: Remaining 20% of dataset
    - **Metrics**: MAE, MSE, RMSE calculated on test set
    """)

# Launch with local URL
demo.launch(server_name="0.0.0.0", server_port=7860)

  Using cached gradio-5.24.0-py3-none-any.whl.metadata (16 kB)
Using cached gradio-5.24.0-py3-none-any.whl (46.9 MB)


/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ad05be1cffe6702cd0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
